# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [54]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [55]:
vac_file = pd.read_csv("../output_data/cities.csv")
vac_file.dropna()
vac_file.head()

,Cloudiness,Country,Date,Maximum Temperature,Humidity,Latitude,Longitude,Wind Speed,City
0,99,RU,1666236076,36.14,72,65.8522,44.2400,18.79,Mezen'
1,6,JP,1666236007,56.32,60,42.3339,142.3669,19.08,Shizunai-furukawachō
2,20,AU,1666235917,68.22,66,-42.8794,147.3294,16.11,Hobart
3,97,CO,1666236077,73.87,84,3.1533,-75.0531,3.83,Baraya
4,82,RU,1666235869,35.49,94,59.7035,30.7861,4.85,Nikolskoye


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [56]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [68]:
# Store "Lat" &"Long" as locations
locations = vac_file[["Latitude", "Longitude"]]

#Store "Humidity" as weight
Humidity = vac_file["Humidity"].astype(float)

In [69]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [70]:
temp_file=vac_file.loc[(vac_file['Maximum Temperature']>=70)&(vac_file['Maximum Temperature']<=80)]
wind_file=temp_file.loc[(temp_file['Wind Speed']<=10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness']==0)]
ideal_weather.head()

,Cloudiness,Country,Date,Maximum Temperature,Humidity,Latitude,Longitude,Wind Speed,City,Nearest Hotel
5,0,US,1666235929,74.95,21,37.1041,-113.5841,4.61,Saint George,Best Western Plus Abbey Inn
20,0,IN,1666235825,77.04,47,28.4667,77.0333,0.00,Gurgaon,Habitare Tamura
105,0,AU,1666236110,77.59,32,-25.2406,130.9889,3.96,Yulara,Desert Gardens Hotel - Ayers Rock Resort
120,0,RE,1666236114,72.91,53,-21.3393,55.4781,4.61,Saint-Pierre,Alize Plage
189,0,BR,1666235901,73.38,94,2.8197,-60.6733,3.44,Boa Vista,Aipana Plaza Hotel


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [71]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(vac_file["City"])):

    lat = vac_file.loc[city]["Latitude"]
    lng = vac_file.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
vac_file["Nearest Hotel"] = hotels
vac_file


,Cloudiness,Country,Date,Maximum Temperature,Humidity,Latitude,Longitude,Wind Speed,City,Nearest Hotel
0,99,RU,1666236076,36.14,72,65.8522,44.2400,18.79,Mezen',Gostevoy Dom Everest
1,6,JP,1666236007,56.32,60,42.3339,142.3669,19.08,Shizunai-furukawachō,Shizunai Eclipse Hotel
2,20,AU,1666235917,68.22,66,-42.8794,147.3294,16.11,Hobart,St Ives Hobart Accommodation Tasmania
3,97,CO,1666236077,73.87,84,3.1533,-75.0531,3.83,Baraya,HOTEL LOS ALMENDROS
4,82,RU,1666235869,35.49,94,59.7035,30.7861,4.85,Nikolskoye,Tourist House - Sablino
...,...,...,...,...,...,...,...,...,...,...
565,100,RU,1666236262,46.99,65,48.4219,42.2308,4.56,Chernyshkovskiy,"The hotel ""Eugenia"""
566,20,US,1666235967,84.18,42,33.8583,-118.0648,6.91,Cerritos,Sheraton Cerritos Hotel
567,0,LY,1666236263,61.05,63,30.3333,10.8500,8.01,Nālūt,Nearest hotel not found
568,100,CO,1666236263,61.16,99,5.4753,-75.2296,2.24,Alejandria,Finca la gran española


In [64]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [65]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

figure.add_layer(markers)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))